### Phase 3: Business & Risk KPIs (1–1.5 Hours) 

#### A. Customer Analytics 

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW customer_count_by_risk_view AS
    SELECT risk_category, COUNT(DISTINCT customer_id) AS customer_count
    FROM silver_customer_account_txn
    GROUP BY risk_category
""")
display(spark.table("customer_count_by_risk_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW avg_annual_income_by_state_view AS
    SELECT state, AVG(annual_income) AS average_annual_income
    FROM silver_customer_account_txn
    GROUP BY state
""")
display(spark.table("avg_annual_income_by_state_view"))

In [0]:
%sql
select * from silver_customer_account_txn

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW customer_age_distribution_view AS
    SELECT 
        FLOOR(DATEDIFF(current_date(), date_of_birth) / 365.25) AS age,
        COUNT(DISTINCT customer_id) AS customer_count
    FROM silver_customer_account_txn
    GROUP BY age
""")
display(spark.table("customer_age_distribution_view"))

In [0]:
%sql
select * from silver_customer_account_txn

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW top_10_customers_by_txn_amount_view AS
    SELECT customer_id, SUM(amount) AS total_transaction_amount
    FROM silver_customer_account_txn
    GROUP BY customer_id
    ORDER BY total_transaction_amount DESC
    LIMIT 10
""")
display(spark.table("top_10_customers_by_txn_amount_view"))

### B. Account Analytics 

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW active_vs_dormant_accounts_view AS
    SELECT 
        CASE 
            WHEN MAX(DATEDIFF(current_date(), last_transaction_date)) <= 100 THEN 'Active'
            ELSE 'Dormant'
        END AS account_status,
        COUNT(DISTINCT account_id) AS account_count
    FROM silver_customer_account_txn
    GROUP BY account_status
""")
display(spark.table("active_vs_dormant_accounts_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW avg_balance_by_account_type_view AS
    SELECT account_type, AVG(current_balance) AS average_balance
    FROM silver_customer_account_txn
    GROUP BY account_type
""")
display(spark.table("avg_balance_by_account_type_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW total_balance_by_branch_view AS
    SELECT branch_code, SUM(current_balance) AS total_balance
    FROM silver_customer_account_txn
    GROUP BY branch_code
""")
display(spark.table("total_balance_by_branch_view"))

In [0]:
MIN_BALANCE_THRESHOLD = 1000  # Set your desired minimum balance threshold

spark.sql(f"""
    CREATE OR REPLACE VIEW accounts_below_min_balance_view AS
    SELECT account_id, current_balance
    FROM silver_customer_account_txn
    WHERE current_balance < {MIN_BALANCE_THRESHOLD}
""")
display(spark.table("accounts_below_min_balance_view"))

### C. Transaction Analytics 

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW monthly_txn_volume_value_view AS
    SELECT 
        YEAR(transaction_date) AS year,
        MONTH(transaction_date) AS month,
        COUNT(*) AS transaction_volume,
        SUM(amount) AS transaction_value
    FROM silver_customer_account_txn
    GROUP BY year, month
    ORDER BY year, month
""")
display(spark.table("monthly_txn_volume_value_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW failed_vs_completed_txn_ratio_view AS
    SELECT
        status,
        COUNT(*) AS transaction_count
    FROM silver_customer_account_txn
    WHERE status IN ('Failed', 'Completed')
    GROUP BY status
""")
display(spark.table("failed_vs_completed_txn_ratio_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW most_frequent_transaction_channels_view AS
    SELECT channel, COUNT(*) AS usage_count
    FROM silver_customer_account_txn
    GROUP BY channel
    ORDER BY usage_count DESC
    LIMIT 10
""")
display(spark.table("most_frequent_transaction_channels_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW merchant_category_contribution_to_revenue_view AS
    SELECT 
        merchant_category,
        SUM(amount) AS total_revenue
    FROM silver_customer_account_txn
    GROUP BY merchant_category
    ORDER BY total_revenue DESC
""")
display(spark.table("merchant_category_contribution_to_revenue_view"))